> __Purpose:__ Implement an initial privacy attack to quantify how much re-identiifcation and linkability risk exists from personalization parameters (the decoder used in the co-adaptation algorithm). Thus, implement a basic ML model to link the decoder matrices back to the corresponding subject.  

-- 7 Subjects, therefore pure guessing would be 14.28% correct on average

In [1]:
import numpy as np
import matplotlib.pyplot as plt
# import seaborn
import time
import pandas as pd
import pickle
import random
from kcs_ml_infr import *

random.seed(a=1)

# Load In Data

In [2]:
keys = ['METACPHS_S106', 'METACPHS_S107','METACPHS_S108', 'METACPHS_S109', 'METACPHS_S110', 'METACPHS_S111', 'METACPHS_S112']

with open('Data\cphs_data_block1.pickle', 'rb') as handle:
    #refs_block1, poss_block1, dec_vels_block1, int_vel_block1, emgs_block1, Ws_block1, Hs_block1, alphas_block1, pDs_block1, times_block1, conditions_block1 = pickle.load(handle)
    _, _, _, _, _, Ws_block1, _, _, _, times_block1, _ = pickle.load(handle)

with open('Data\cphs_data_block2.pickle', 'rb') as handle:
    #refs_block2, poss_block2, dec_vels_block2, int_vel_block2, emgs_block2, Ws_block2, Hs_block2, alphas_block2, pDs_block2, times_block2, conditions_block2 = pickle.load(handle)
    _, _, _, _, _, Ws_block2, _, _, _, times_block2, _ = pickle.load(handle)

In [3]:
# Decoder updates
W = Ws_block1[keys[0]][0]
W[1:,:,:].shape # 7199 time points x (decoder dimensions is 2 x 6)
dold = W[0]
update_ix = []
for ix,d in enumerate(W[1:]):
  if (np.array_equal(dold,d)==False):
    update_ix.append(ix)
    dold = d

update_ix.append(len(W) - 1) 
update_ix = np.asarray(update_ix)
update_ix = np.hstack([[0],update_ix])

print(update_ix.shape)
print(update_ix)

(19,)
[    0  1200  2402  3604  4806  6008  7210  8412  9614 10816 12018 13220
 14422 15624 16826 18028 19230 20432 20769]


I'm just making the flatten array input DF here

In [4]:
dec_flattened_df1 = pd.DataFrame(columns=["Subject", "Condition", "Update Number", "Flattened Decoder"])
dec_flattened_df2 = pd.DataFrame(columns=["Subject", "Condition", "Update Number", "Flattened Decoder"])

dec_flattened_df1.head()

,Subject,Condition,Update Number,Flattened Decoder


In [5]:
num_conds = 8

t0 = time.time()

for key in keys:
    participant_dec1 = Ws_block1[key]
    participant_dec2 = Ws_block2[key]
    
    for my_cond in range(num_conds):
        for update_number, update_idx in enumerate(update_ix):
            dec_flattened_df1.loc[len(dec_flattened_df1)] = [key, my_cond, update_number, np.ravel(participant_dec1[my_cond, update_idx, :, :])]
            dec_flattened_df2.loc[len(dec_flattened_df2)] = [key, my_cond, update_number, np.ravel(participant_dec2[my_cond, update_idx, :, :])]
        
t1 = time.time()
total = t1-t0  
print(total)

3.9847707748413086


Also make a DF of vector of norms (wrt ending point)

In [6]:
#dec_vec_norm_df1 = pd.DataFrame(columns=["Subject", "Condition", "Norm Vector"])
#dec_vec_norm_df2 = pd.DataFrame(columns=["Subject", "Condition", "Norm Vector"])

#dec_vec_norm_df1.head()

In [7]:
# Not working like it's supposed to 
'''
#
dec_per_cond_b1 = [0]*num_conds
dec_per_cond_b2 = [0]*num_conds
#
norm_per_cond_b1 = [0]*num_conds
norm_per_cond_b2 = [0]*num_conds
for cond in range(num_conds):  
    dec_per_cond_b1[cond] = [0]*len(update_ix)
    dec_per_cond_b2[cond] = [0]*len(update_ix)
    #
    norm_per_cond_b1[cond] = [0]*len(update_ix)
    norm_per_cond_b2[cond] = [0]*len(update_ix)

for key in keys:
    participant_dec1 = Ws_block1[key]
    participant_dec2 = Ws_block2[key]
    
    for my_cond in range(num_conds):
        for idx, my_update in enumerate(update_ix):
            dec_per_cond_b1[my_cond][idx] = participant_dec1[my_cond, my_update, :, :]
            dec_per_cond_b2[my_cond][idx] = participant_dec2[my_cond, my_update, :, :]
    
#################################
for my_cond in range(num_conds):
    for dec_num in range(len(update_ix)):
        norm_per_cond_b1[my_cond][dec_num] = np.linalg.norm(dec_per_cond_b1[my_cond][dec_num] - dec_per_cond_b1[my_cond][-1])
        norm_per_cond_b2[my_cond][dec_num] = np.linalg.norm(dec_per_cond_b2[my_cond][dec_num] - dec_per_cond_b2[my_cond][-1])
        
#################################
for key in keys:
    participant_dec1 = Ws_block1[key]
    participant_dec2 = Ws_block2[key]
    
    for my_cond in range(num_conds):
        for idx, my_update in enumerate(update_ix):
            dec_vec_norm_df1.loc[len(dec_vec_norm_df1)] = [key, my_cond, norm_per_cond_b1[my_cond]]
            dec_vec_norm_df2.loc[len(dec_vec_norm_df2)] = [key, my_cond, norm_per_cond_b2[my_cond]]
'''
0

0

Load in the other data
> This cell takes forever to run... not sure if it was actually any quicker to import vs just make it all over again

In [8]:
t0 = time.time()

dec_norms_df1 = pd.read_csv("Data\decoder_norms1.csv")
dec_norms_df2 = pd.read_csv("Data\decoder_norms2.csv")

t1 = time.time()
total = t1-t0  
print(total)

0.011649608612060547


In [9]:
dec_norms_df = pd.concat((dec_norms_df1, dec_norms_df2))
dec_flattened_df = pd.concat((dec_flattened_df1, dec_flattened_df2))
# This one isn't working yet
#dec_vec_norm_df = pd.concat((dec_vec_norm_df1, dec_vec_norm_df2))

In [10]:
print(dec_norms_df.shape)
dec_norms_df.drop('Unnamed: 0', axis=1, inplace=True)
dec_norms_df.head()

(2128, 5)


,Subject,Condition,Update Number,Frobenius Norm
0,METACPHS_S106,0,0,0.066360
1,METACPHS_S106,0,1,0.066360
2,METACPHS_S106,0,2,9.709390
3,METACPHS_S106,0,3,8.209080
4,METACPHS_S106,0,4,10.406943


In [11]:
print(dec_flattened_df.shape)
dec_flattened_df.head()

(2128, 4)


,Subject,Condition,Update Number,Flattened Decoder
0,METACPHS_S106,0,0,"[0.002722144351611262, 0.002605931562722017, 0..."
1,METACPHS_S106,0,1,"[0.002722144351611262, 0.002605931562722017, 0..."
2,METACPHS_S106,0,2,"[-0.34157085409453486, 0.08410593293763585, -0..."
3,METACPHS_S106,0,3,"[-0.1738624752800762, 0.003705171262358347, 0...."
4,METACPHS_S106,0,4,"[0.8321061501817386, -1.3261242289666402, 0.28..."


# Classification
1. Logistic Regression
2. K-Nearest Neighbor
3. Gaussian Naive Bayes
4. Linear SVC
5. Stochastic Gradient Descent
6. Decision Tree Classifier
7. Gradient Boosting Trees

In [12]:
# Machine learning
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [13]:
# Set the number of k-folds
cv = 10

## 1) Linking Matrix Norms to Participants

In [14]:
norm_label_df = dec_norms_df.drop(["Condition", "Update Number", "Frobenius Norm"], axis=1)

In [15]:
key_to_num = dict()
num_to_key = dict()
for idx, key in enumerate(keys):
    key_to_num[key] = idx
    num_to_key[idx] = key

In [16]:
norm_label_df["Subject"] = norm_label_df["Subject"].map(key_to_num)
norm_label_df.head()

,Subject
0,0
1,0
2,0
3,0
4,0


In [17]:
# Result logs
my_metrics_cols = ['Algorithm', 'One Off Acc', 'CV Acc', 'K Folds']
res_df = pd.DataFrame(columns=my_metrics_cols)

norm1d_res_df = pd.DataFrame(columns=my_metrics_cols)
SSnorm1d_res_df = pd.DataFrame(columns=my_metrics_cols)
flatten_dec_res_df = pd.DataFrame(columns=my_metrics_cols)
SSflatten_dec_res_df = pd.DataFrame(columns=my_metrics_cols)
norm_vec_res_df = pd.DataFrame(columns=my_metrics_cols)
SSnorm_vec_res_df = pd.DataFrame(columns=my_metrics_cols)

In [18]:
my_models = [LogisticRegression(), KNeighborsClassifier(), GaussianNB(), LinearSVC(), SGDClassifier(), DecisionTreeClassifier(), GradientBoostingClassifier()]

## Case 1: 1D Norm

In [19]:
norm_input_df = dec_norms_df.drop(["Subject",  "Condition", "Update Number"], axis=1)
norm_input_df.head()

,Frobenius Norm
0,0.066360
1,0.066360
2,9.709390
3,8.209080
4,10.406943


In [20]:
X_train, y_train, X_test, y_test, X_val, y_val = train_test_val_split(norm_input_df, norm_label_df)
y_train = np.ravel(y_train)

print(X_train.shape)
X_train.head()

(1489, 1)


,Frobenius Norm
420,3.567582
146,5.589772
266,0.069974
695,5.796815
372,17.938752


In [21]:
for model_num, model in enumerate(my_models):
    norm1d_res_df = train_model(model, X_train, y_train, cv, norm1d_res_df)

C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [22]:
norm1d_res_df.head(100)

,Algorithm,One Off Acc,CV Acc,K Folds
0,LogisticRegression(),17.931,16.857,10
0,KNeighborsClassifier(),52.048,30.356,10
0,GaussianNB(),19.073,17.260,10
0,LinearSVC(),19.208,19.208,10
0,SGDClassifier(),14.574,14.909,10
0,DecisionTreeClassifier(),100.000,28.946,10
0,GradientBoostingClassifier(),65.682,32.304,10


Now Test

In [23]:
test_df = pd.DataFrame(columns=['Algorithm', 'CV Acc', 'Test Acc', 'K Folds'])

for model_num, model in enumerate(my_models):
    test_df = test_model(model, X_train, y_train, X_test, y_test, test_df, cv)

C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepd

In [24]:
test_df.head(100)

,Algorithm,CV Acc,Test Acc,K Folds
0,LogisticRegression(),16.857,16.901,10
0,KNeighborsClassifier(),30.356,28.795,10
0,GaussianNB(),17.26,19.718,10
0,LinearSVC(),18.267,22.066,10
0,SGDClassifier(),15.312,12.833,10
0,DecisionTreeClassifier(),28.946,27.856,10
0,GradientBoostingClassifier(),32.304,27.23,10


## Case 2: Flattened Decoder Matrices as Input

In [25]:
flattened_input_df = dec_flattened_df.drop(["Subject",  "Condition", "Update Number"], axis=1)
flattened_input_df.head()

,Flattened Decoder
0,"[0.002722144351611262, 0.002605931562722017, 0..."
1,"[0.002722144351611262, 0.002605931562722017, 0..."
2,"[-0.34157085409453486, 0.08410593293763585, -0..."
3,"[-0.1738624752800762, 0.003705171262358347, 0...."
4,"[0.8321061501817386, -1.3261242289666402, 0.28..."


In [26]:
X_train, y_train, X_test, y_test, X_val, y_val = train_test_val_split(norm_input_df, norm_label_df)
y_train = np.ravel(y_train)

print(X_train.shape)
X_train.head()

(1489, 1)


,Frobenius Norm
420,3.567582
146,5.589772
266,0.069974
695,5.796815
372,17.938752


Convert single element of a vector into a vector of single elements!

In [27]:
# Idk what happened causing me to not need this code anymore...

X_train2 = X_train
X_test2 = X_test

'''
X_train2 = pd.DataFrame()
X_test2 = pd.DataFrame()
for my_row in range(X_train.shape[0]):
    test=pd.DataFrame(X_train.iloc[my_row,0]).T
    X_train2 = pd.concat((X_train2, test))
    
for my_row in range(X_test.shape[0]):
    test=pd.DataFrame(X_test.iloc[my_row,0]).T
    X_test2 = pd.concat((X_test2, test))
'''

X_train2.head()

,Frobenius Norm
420,3.567582
146,5.589772
266,0.069974
695,5.796815
372,17.938752


In [28]:
for model_num, model in enumerate(my_models):
    print(f"{model_num} of {len(my_models)}")
    flatten_dec_res_df = train_model(model, X_train2, y_train, cv, flatten_dec_res_df)

0 of 7
1 of 7


C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


2 of 7
3 of 7


C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


4 of 7
5 of 7
6 of 7


In [29]:
flatten_dec_res_df.head(100)

,Algorithm,One Off Acc,CV Acc,K Folds
0,LogisticRegression(),17.931,16.857,10
0,KNeighborsClassifier(),52.048,30.356,10
0,GaussianNB(),19.073,17.260,10
0,LinearSVC(),19.275,18.737,10
0,SGDClassifier(),14.171,14.506,10
0,DecisionTreeClassifier(),100.000,28.946,10
0,GradientBoostingClassifier(),65.682,32.304,10


Test the models on the testing data

In [30]:
test_df = pd.DataFrame(columns=['Algorithm', 'CV Acc', 'Test Acc', 'K Folds'])
for model in my_models:
    test_df = test_model(model, X_train2, y_train, X_test2, y_test, test_df, cv, verbose=True)

LogisticRegression()
CV Accuracy: 16.857
Test Accuracy: 16.901

KNeighborsClassifier()
CV Accuracy: 30.356
Test Accuracy: 28.795

GaussianNB()
CV Accuracy: 17.26
Test Accuracy: 19.718



C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepd

LinearSVC()
CV Accuracy: 18.334
Test Accuracy: 19.718

SGDClassifier()
CV Accuracy: 14.238
Test Accuracy: 11.737

DecisionTreeClassifier()
CV Accuracy: 28.946
Test Accuracy: 27.856

GradientBoostingClassifier()
CV Accuracy: 32.304
Test Accuracy: 27.23



In [31]:
test_df.head(100)

,Algorithm,CV Acc,Test Acc,K Folds
0,LogisticRegression(),16.857,16.901,10
0,KNeighborsClassifier(),30.356,28.795,10
0,GaussianNB(),17.26,19.718,10
0,LinearSVC(),18.334,19.718,10
0,SGDClassifier(),14.238,11.737,10
0,DecisionTreeClassifier(),28.946,27.856,10
0,GradientBoostingClassifier(),32.304,27.23,10


Redo but try using Standard Scaler this time

In [32]:
from sklearn.preprocessing import StandardScaler

scaler_Xtrain2 = StandardScaler().fit(X_train2)
Xtrain2SS = scaler_Xtrain2.transform(X_train2)

print(X_train2.shape)
print(Xtrain2SS.shape)

(1489, 1)
(1489, 1)


In [33]:
for model_num, model in enumerate(my_models):
    print(f"{model_num+1} of {len(my_models)}")
    SSflatten_dec_res_df = train_model(model, Xtrain2SS, y_train, cv, SSflatten_dec_res_df)

1 of 7
2 of 7
3 of 7
4 of 7


C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


5 of 7
6 of 7
7 of 7


In [34]:
SSflatten_dec_res_df.head(100)

,Algorithm,One Off Acc,CV Acc,K Folds
0,LogisticRegression(),17.999,16.857,10
0,KNeighborsClassifier(),52.048,30.356,10
0,GaussianNB(),19.073,17.260,10
0,LinearSVC(),19.275,18.267,10
0,SGDClassifier(),18.267,14.909,10
0,DecisionTreeClassifier(),100.000,28.946,10
0,GradientBoostingClassifier(),65.682,32.304,10


Thus we see no real performance gains by using standard scaler.

Now try and optimize the best performer (GBT)
1. max_depth: int, default=3
- The maximum depth of the individual regression estimators. The maximum depth limits the number of nodes in the tree. Tune this parameter for best performance; the best value depends on the interaction of the input variables. Values must be in the range [1, inf).

## Case 3: Feed in a Vector of Norms

In [35]:
assert(1==0)

AssertionError: 

In [ ]:
#dec_vec_norm_df
dec_vec_norm_input_df = dec_vec_norm_df.drop(["Subject",  "Condition"], axis=1)
dec_vec_norm_input_df.head()

norm_label_df probably needs to be changed to a new variable... shouldn't have the same shape...

In [ ]:
X_train, y_train, X_test, y_test, X_val, y_val = train_test_val_split(norm_input_df, norm_label_df)
y_train = np.ravel(y_train)

print(X_train.shape)
X_train.head()

In [ ]:
X_train2 = pd.DataFrame()
X_test2 = pd.DataFrame()
for my_row in range(X_train.shape[0]):
    test=pd.DataFrame(X_train.iloc[my_row,0]).T
    X_train2 = pd.concat((X_train2, test))
    
for my_row in range(X_test.shape[0]):
    test=pd.DataFrame(X_test.iloc[my_row,0]).T
    X_test2 = pd.concat((X_test2, test))
X_train2.head()

In [ ]:
for model_num, model in enumerate(my_models):
    print(f"{model_num} of {len(my_models)}")
    norm_vec_res_df = train_model(model, X_train2, y_train, cv, norm_vec_res_df)

In [ ]:
norm_vec_res_df.head(100)

## 2) Cluster Decoder Matrix Norms
> Cluster using 2D dataset of (norm, condition), condition being the y axis maybe?
> Still not sure if I should just take the norm of take the norm wrt the first/last.  
> Can I make a similarity matrix? How would I use that for clustering...
> Save this idea for later... clustering is just another form of an attack vector, the ML model above already was successful